In [1]:
import torch
torch.set_default_tensor_type(torch.FloatTensor)
from torch.autograd import Variable
from torch.nn import functional as F
from torchmetrics import Accuracy, Recall, Precision, Specificity, ConfusionMatrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_recall_curve, average_precision_score,roc_curve, auc, precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt
from collections import Counter
import random
import math
import torch.optim as optim
from tabulate import tabulate
from ray import tune

from fractions import Fraction

In [2]:
# Create Tensors to hold input and outputs.
beta_1=np.append(np.array([4,4,-8,-4,6,8,-4,4,-6,-8]),np.zeros(90))
beta_2=np.append(np.array([-2,2,4,-2,3,-4,2,-2,-3,4]),np.zeros(90))
beta_3=np.append(np.array([-3,-3,6,-3,-4.5,-6,3,-3,4.5,6]),np.zeros(90))
beta_1=beta_1.reshape(-1,1)
beta_2=beta_2.reshape(-1,1)
beta_3=beta_3.reshape(-1,1)

n1=200
n2=200
n3=200
p=100
corval=0
prior=[1,2,3,4,5]

def sigmoid(X):
    return .5 * (1 + np.tanh(.5 * X))

def generate_data(corval,beta,n,p):
    mean=np.zeros(p)
    sigma=np.array([[corval**abs(i-j) for i in range(p)] for j in range(p)])
    x=torch.tensor(np.random.multivariate_normal(mean=mean,cov=sigma,size=n))
    Pi_test= sigmoid(x@beta)
    y=np.random.binomial(1,Pi_test.ravel(),n)
    x=x.to(torch.float32)
    #y=torch.tensor(y,dtype=torch.double)
    y=torch.tensor(y,dtype=torch.float)
    #y=y.to(torch.float32)
    return x,y
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

def cubic_root(x):
    return math.copysign(math.pow(abs(x), 1.0/3.0), x)

In [3]:
def train_model(config):
    split_num=5
    kf = KFold(n_splits=split_num)



    sum_test_BCE_loss=0
    for idx_train,idx_test in kf.split(inputs):
        lambda1,lambda2,lr,ga= config["lambda1"], config["lambda2"],config["lr"],config["ga"]
        model=MLP(seed=1)
        max_iteration=1000
        learning_rate=lr
        params_to_optimize=list(model.parameters())
        optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
        loss_fn = torch.nn.BCELoss()
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=ga)

        BCE=[]

        for t in range(max_iteration):
            optimizer.zero_grad() # renew optimizer
            out, layer1_out, layer2_out= model(inputs[idx_train])

            # extract parameters
            #[:-1] for leaving out bias term#
            model_all_linear1_params = model.linear1
            model_all_linear2_params= torch.cat([x.view(-1) for x in model.linear2.parameters()][:-1])
            model_all_linear3_params= torch.cat([x.view(-1) for x in model.linear3.parameters()][:-1])

    


            # compute loss
            all_linear1_params=model_all_linear1_params.reshape(1,-1)
            all_linear23_params=torch.cat((model_all_linear2_params,model_all_linear3_params),0)
            
            non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
            BCE_loss = loss_fn(out, targets[idx_train].reshape(-1,1))


#             l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)
            l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

            l2_regularization = lambda2* torch.sum(all_linear23_params.pow(2))

            loss =BCE_loss+l1_regularization +l2_regularization

            # record loss
            BCE.append(BCE_loss.item())

            # compute derivative
            loss.backward()

            # gradient descent
            optimizer.step()

            # learning rate decay
            scheduler.step()
        
        
        test_out, test_layer1_out, test_layer2_out= model(train_x[idx_test])

        
        test_BCE_loss = loss_fn(test_out, targets[idx_test].reshape(-1,1))

        
        sum_test_BCE_loss+=test_BCE_loss.item()

    #print("loss: ",)
    tune.report(my_test_BCE_loss=sum_test_BCE_loss/split_num)

In [4]:
def train_model_2(config):
    split_num=5
    kf = KFold(n_splits=split_num)
    
    
    lambda1,lambda2,lr,eta = config["lambda1"], config["lambda2"],config["lr"],config["eta"]
    inputs=train_x
    targets=(1-eta)*train_y+eta*y_prior
    
    
    sum_test_BCE_loss=0
    for idx_train,idx_test in kf.split(inputs):
       
        
        model=MLP(seed=1)

        max_iteration=1000
        learning_rate=lr
        params_to_optimize=list(model.parameters())
        optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
        loss_fn = torch.nn.BCELoss()
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

        BCE=[]

        for t in range(max_iteration):
            optimizer.zero_grad() # renew optimizer
            out, layer1_out, layer2_out,layer3_out= model(inputs[idx_train])


            # extract parameters
            #[:-1] for leaving out bias term#
            model_all_linear1_params = model.linear1
            model_all_linear2_params= torch.cat([x.view(-1) for x in model.linear2.parameters()][:-1])
            model_all_linear3_params= torch.cat([x.view(-1) for x in model.linear3.parameters()][:-1])
            model_all_linear4_params= torch.cat([x.view(-1) for x in model.linear4.parameters()][:-1])
    


            # compute loss
            all_linear1_params=model_all_linear1_params.reshape(1,-1)
            all_linear234_params=torch.cat((model_all_linear2_params,model_all_linear3_params, model_all_linear4_params),0)

            BCE_loss = loss_fn(out, targets[idx_train].reshape(-1,1))
            
            non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
#             l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)
            l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

            l2_regularization = lambda2* torch.sum(all_linear234_params.pow(2))
            loss =BCE_loss+l1_regularization +l2_regularization

            # record loss
            BCE.append(BCE_loss.item())

            # compute derivative
            loss.backward()

            # gradient descent
            optimizer.step()

            # learning rate decay
            scheduler.step()
        
        
        test_out, test_layer1_out, test_layer2_out,test_layer3_out= model(train_x[idx_test])

        test_BCE_loss = loss_fn(test_out, targets[idx_test].reshape(-1,1))
        
        sum_test_BCE_loss+=test_BCE_loss.item()


    #print("loss: ",)
    tune.report(my_test_BCE_loss=sum_test_BCE_loss/split_num)

In [5]:
TPR=[]
TNR=[]
GM=[]
accuracy=[]
precision=[]
f1=[]=[]
vsSEN=[]
vsSPE=[]
vsGM=[]
vsMR=[]
vsCCR=[]


vs = np.zeros((1,100))

In [6]:
for i in range(5):
    setup_seed(i)
       
    x1,y1=generate_data(corval,beta_1,n1,p)
    x2,y2=generate_data(corval,beta_2,n2,p)
    x3,y3=generate_data(corval,beta_3,n3,p)

    x= torch.cat((x1,x2), dim=0)
    x= torch.cat((x,x3), dim=0)

    y= torch.cat((y1,y2), dim=0)
    y= torch.cat((y,y3), dim=0)
    
    train_x, test_x, train_y, test_y= train_test_split(x,y, test_size=0.2, random_state=1)

    
    # switch training set
    inputs,targets = train_x,train_y

    # switching testing set
    test_inputs, test_targets = test_x,test_y
    
################## first step######################################  
    prior=[1,2,3,4,5]

# choose lambda1,lambda2 by ray tune
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1 = torch.nn.Parameter(torch.randn(p))
            self.linear2 = torch.nn.Linear(p,10)
            self.linear3 = torch.nn.Linear(10,1)

        def forward(self, x):
            layer1_out = self.linear1*x
            layer2_out = F.relu(self.linear2(layer1_out))
            out= torch.sigmoid(self.linear3(layer2_out))
            return out, layer1_out, layer2_out


    config={
            "lr":tune.choice([0.6,0.7,0.8]),
            "lambda1": tune.choice([0.03,0.04,0.05]),
            "lambda2": tune.choice([0.001,0.002,0.003]),
            "ga":tune.choice([0.3,0.4,0.5])
        }

    result = tune.run(
            train_model,  
            config=config,
            num_samples=30)
    
################## second step########################################    

    final_lambda=result.get_best_config('my_test_BCE_loss',mode='min')
    lambda1,lambda2,lr,ga=final_lambda["lambda1"],final_lambda["lambda2"],final_lambda["lr"],final_lambda["ga"]
    learning_rate=lr
    max_iteration=1000
    
    BCE=[]
    
    model=MLP(seed=1)

    loss_fn = torch.nn.BCELoss()
    params_to_optimize=list(model.parameters())
    optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
    
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=ga)

    loss_record=[]
    
    for t in range(max_iteration):
    
        # renew optimizer
        optimizer.zero_grad()

        # forward propagate
        out, layer1_out, layer2_out= model(inputs)



        # extract parameters
        #[:-1] for leaving out bias term#
        model_all_linear1_params = model.linear1
        model_all_linear2_params= torch.cat([x.view(-1) for x in model.linear2.parameters()][:-1])
        model_all_linear3_params= torch.cat([x.view(-1) for x in model.linear3.parameters()][:-1])

        # compute loss

        all_linear1_params=model_all_linear1_params.reshape(1,-1)
        all_linear23_params=torch.cat((model_all_linear2_params,model_all_linear3_params),0)

        BCE_loss = loss_fn(out, targets.reshape(-1,1))
        
        non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
    #     l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)

        l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

        l2_regularization = lambda2 * torch.sum(all_linear23_params.pow(2))

        loss =BCE_loss+l1_regularization +l2_regularization

        loss_record.append(loss.item())
        # record loss
        BCE.append(BCE_loss.item())

        # compute derivative
        loss.backward()

        # gradient descent
        optimizer.step()

        # learning rate decay
        scheduler.step()
    
################## Y_prior##################    
    y_prior= (model(train_x)[0]).detach().squeeze()

    
    
################## third step##################

    #no prior
    prior=[]
    # choose lambda1,lambda2 by ray tune
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1 = torch.nn.Parameter(torch.randn(p))
            self.linear2 = torch.nn.Linear(p,10)
            self.linear3 = torch.nn.Linear(10,10)
            self.linear4 = torch.nn.Linear(10,1)
        def forward(self, x):
            layer1_out = self.linear1*x
            layer2_out = F.relu(self.linear2(layer1_out))
            layer3_out = F.relu(self.linear3(layer2_out))
            out= torch.sigmoid(self.linear4(layer3_out))
            return out, layer1_out, layer2_out,layer3_out

    config={
        "lambda1": tune.choice([0.03,0.04,0.05]),
        "lambda2": tune.choice([0.001,0.002,0.003]),
         "eta": tune.choice([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]),
         "lr":tune.choice([0.6,0.7,0.8])
                            }

    result = tune.run(
            train_model_2,  
            config=config,
            num_samples=150)
    

################## fourth step##################

    #no prior
    prior=[]
    #0.4,0.4
    # set hyperparamters
    final_parameter=result.get_best_config('my_test_BCE_loss',mode='min')
    lambda1,lambda2,eta,lr=final_parameter["lambda1"],final_parameter["lambda2"],final_parameter["eta"],final_parameter["lr"]
    learning_rate=lr
    max_iteration=1000

    inputs=train_x
    targets=(1-eta)*train_y+eta*y_prior

    # switching testing set
    test_inputs, test_targets = test_x,test_y

    # record loss descent
    BCE=[]


    # main nn object
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1 = torch.nn.Parameter(torch.randn(p))
            self.linear2 = torch.nn.Linear(p,10)
            self.linear3 = torch.nn.Linear(10,10)
            self.linear4 = torch.nn.Linear(10,1)

        def forward(self, x):
            layer1_out = self.linear1*x
            layer2_out = F.relu(self.linear2(layer1_out))
            layer3_out = F.relu(self.linear3(layer2_out))
            out= torch.sigmoid(self.linear4(layer3_out))
            return out, layer1_out, layer2_out,layer3_out



    model=MLP(seed=1)

    loss_fn = torch.nn.BCELoss()
    params_to_optimize=list(model.parameters())
    optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)

    # learning rate decay scheme
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

    loss_record=[]
    # loop for max_iteration times
    for t in range(max_iteration):

        # renew optimizer
        optimizer.zero_grad()

        # forward propagate
        out, layer1_out, layer2_out,layer3_out= model(inputs)



        model_all_linear1_params = model.linear1
        model_all_linear2_params= torch.cat([x.view(-1) for x in model.linear2.parameters()][:-1])
        model_all_linear3_params= torch.cat([x.view(-1) for x in model.linear3.parameters()][:-1])
        model_all_linear4_params= torch.cat([x.view(-1) for x in model.linear4.parameters()][:-1])

        # compute loss

        all_linear1_params=model_all_linear1_params.reshape(1,-1)
        all_linear234_params=torch.cat((model_all_linear2_params,model_all_linear3_params, model_all_linear4_params),0)

        BCE_loss = loss_fn(out, targets.reshape(-1,1))

        non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)

        l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))
        l2_regularization = lambda2 * torch.sum(all_linear234_params.pow(2))
        loss =BCE_loss+l1_regularization +l2_regularization

        loss_record.append(loss.item())
        # record loss
        BCE.append(BCE_loss.item())

        # compute derivative
        loss.backward()

        # gradient descent
        optimizer.step()

        # learning rate decay
        scheduler.step()
        
    best_trial = result.get_best_trial("my_test_BCE_loss", "min", "last")
    final_parameter=result.get_best_config('my_test_BCE_loss',mode='min')
    print(final_parameter)
    
####################################   prediction ####################################      

    prediction= (model(test_x)[0]>0.5).clone().int()
    target= test_y.reshape(-1,1).int()


    prediction=prediction.tolist()
    target=target.tolist()

    final_matrix=confusion_matrix(target,prediction)
    TPR.append(recall_score(target,prediction))#TPR
    TNR.append(final_matrix[0,0]/(sum(final_matrix[0,:])))#TNR
    GM.append(math.sqrt(TPR[-1]*TNR[-1]))
    accuracy.append(accuracy_score(target,prediction))
    precision.append(precision_score(target,prediction))
    f1.append(f1_score(target,prediction))

    
    
####################################   variable selection ####################################     
    model_weight=model_all_linear1_params.detach().numpy().copy().reshape(1,-1)

    threshold=0.12
    mcl_w=np.zeros(p)

    for j in range(p):
        mcl_w[j]=model_weight[0,j]

    max_mcl_w=max(abs(mcl_w))
    for j in range(p):
        if (abs(mcl_w[j]))<=threshold*max_mcl_w:
            model_weight[0,j]=0
        else:
            model_weight[0,j]=model_weight[0,j]
    
    index=np.nonzero(model_weight)[1]
    
    beta=np.array(beta_1)
    trueindex=np.where(beta!=0)[0]
    trueindex=trueindex.tolist()
    trueindex=set(trueindex)
    TP=len(trueindex.intersection(index))
    FP=len(np.setdiff1d(index,trueindex))
    FN=len(trueindex)-TP
    TN=p-len(trueindex)-FP
    
    vsSEN.append(TP/(TP+FN))
    vsSPE.append(TN/(TN+FP))
    vsGM.append(math.sqrt(vsSEN[-1]*vsSPE[-1]))
    vsMR.append((FP+FN)/(TP+FN+TN+FP))
    vsCCR.append(1-vsMR[-1])
    
    print(i)

2022-11-02 18:03:40,578	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8272 
2022-11-02 18:03:42,174	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,ga,lambda1,lambda2,lr,iter,total time (s),my_test_BCE_loss
train_model_a157d_00000,TERMINATED,10.224.0.12:5466,0.3,0.03,0.003,0.8,1,12.0316,0.709613
train_model_a157d_00001,TERMINATED,10.224.0.12:6632,0.5,0.03,0.001,0.8,1,20.9661,0.767686
train_model_a157d_00002,TERMINATED,10.224.0.12:6635,0.4,0.05,0.001,0.6,1,17.649,0.673449
train_model_a157d_00003,TERMINATED,10.224.0.12:6636,0.4,0.05,0.002,0.8,1,18.9174,0.669645
train_model_a157d_00004,TERMINATED,10.224.0.12:6638,0.4,0.04,0.002,0.7,1,18.5174,0.679179
train_model_a157d_00005,TERMINATED,10.224.0.12:6640,0.4,0.03,0.002,0.8,1,20.5308,0.727507
train_model_a157d_00006,TERMINATED,10.224.0.12:6642,0.4,0.04,0.002,0.7,1,19.1079,0.679179
train_model_a157d_00007,TERMINATED,10.224.0.12:6644,0.4,0.03,0.001,0.7,1,19.5834,0.7389
train_model_a157d_00008,TERMINATED,10.224.0.12:6646,0.5,0.03,0.003,0.8,1,18.6367,0.723799
train_model_a157d_00009,TERMINATED,10.224.0.12:6648,0.5,0.03,0.001,0.6,1,17.1571,0.743938


(raylet) [2022-11-02 18:03:49,506 E 4449 4482] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_18-03-36_855327_2667 is over 95% full, available space: 377450496; capacity: 29394726912. Object creation will fail if spilling is required.


Result for train_model_a157d_00000:
  date: 2022-11-02_18-03-56
  done: false
  experiment_id: 420beddfd580434caff8975ccdc5ae07
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.7096134066581726
  node_ip: 10.224.0.12
  pid: 5466
  time_since_restore: 12.031636238098145
  time_this_iter_s: 12.031636238098145
  time_total_s: 12.031636238098145
  timestamp: 1667383436
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: a157d_00000
  warmup_time: 0.00383758544921875
  
Result for train_model_a157d_00000:
  date: 2022-11-02_18-03-56
  done: true
  experiment_id: 420beddfd580434caff8975ccdc5ae07
  experiment_tag: 0_ga=0.3000,lambda1=0.0300,lambda2=0.0030,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.7096134066581726
  node_ip: 10.224.0.12
  pid: 5466
  time_since_restore: 12.031636238098145
  time_this_iter_s: 12.031636238098145
  time_total_s: 12.031636238098145
  timestamp: 1667383436
  timestep

(raylet) [2022-11-02 18:03:59,577 E 4449 4482] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_18-03-36_855327_2667 is over 95% full, available space: 375885824; capacity: 29394726912. Object creation will fail if spilling is required.


...

Result for train_model_2_113dd_00138:
  date: 2022-11-02_18-23-20
  done: false
  experiment_id: 99c9aa0295fb46d9a5f965a50e0b51b2
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.6936204552650451
  node_ip: 10.224.0.12
  pid: 13153
  time_since_restore: 20.037240743637085
  time_this_iter_s: 20.037240743637085
  time_total_s: 20.037240743637085
  timestamp: 1667384600
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 113dd_00138
  warmup_time: 0.015192270278930664
  
Result for train_model_2_113dd_00138:
  date: 2022-11-02_18-23-20
  done: true
  experiment_id: 99c9aa0295fb46d9a5f965a50e0b51b2
  experiment_tag: 138_eta=0.5000,lambda1=0.0500,lambda2=0.0030,lr=0.7000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.6936204552650451
  node_ip: 10.224.0.12
  pid: 13153
  time_since_restore: 20.037240743637085
  time_this_iter_s: 20.037240743637085
  time_total_s: 20.037240743637085
  timestamp: 1667384600


Result for train_model_2_113dd_00143:
  date: 2022-11-02_18-23-22
  done: false
  experiment_id: 7105702e3afd44d386fdb0e9988fd722
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.693392550945282
  node_ip: 10.224.0.12
  pid: 13133
  time_since_restore: 17.439598560333252
  time_this_iter_s: 17.439598560333252
  time_total_s: 17.439598560333252
  timestamp: 1667384602
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 113dd_00143
  warmup_time: 0.012956619262695312
  
Result for train_model_2_113dd_00143:
  date: 2022-11-02_18-23-22
  done: true
  experiment_id: 7105702e3afd44d386fdb0e9988fd722
  experiment_tag: 143_eta=0.6000,lambda1=0.0400,lambda2=0.0010,lr=0.7000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.693392550945282
  node_ip: 10.224.0.12
  pid: 13133
  time_since_restore: 17.439598560333252
  time_this_iter_s: 17.439598560333252
  time_total_s: 17.439598560333252
  timestamp: 1667384602
  

2022-11-02 18:23:24,676	INFO tune.py:758 -- Total run time: 135.80 seconds (135.38 seconds for the tuning loop).


{'lambda1': 0.05, 'lambda2': 0.003, 'eta': 0.9, 'lr': 0.6}
4


/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
####################################   prediction ####################################   
from numpy import *
mean(TPR)
mean(TNR)
mean(GM)
mean(accuracy)
mean(precision)
mean(f1)


0.3991937116289531

0.7122794698267233

0.4376033287156744

0.5666666666666667

0.46411869211869217

0.417098285139163

In [8]:
####################################   variable selection ####################################   
mean(vsSEN)
mean(vsSPE)
mean(vsGM)
mean(vsMR)
mean(vsCCR)

0.27999999999999997

0.7866666666666667

0.32102529672369284

0.264

0.736

In [20]:
vs

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [21]:
i

4